## Class Activation Map Visualization

### Visualization Function

In [9]:
# Import Statements

from keras.models import *
import keras.backend as K
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Dense, Lambda
import os


#Visualization
def visualize_class_activation_map(model, test_img_path,output_path,target_class):
        #Read original image
        original_img = cv2.imread(test_img_path, 1)
        
        #Resize image to pass through the model (not necessary)
        img = cv2.resize(original_img,(150,150))
        width, height, _ = img.shape

        #Expand dims to create a batch of size 1
        batch_img = np.expand_dims(img,axis=0)
        
        #Get weights to the final dense layer feeding to softmax
        class_weights = model.layers[-1].get_weights()[0]
        
        #Get final convolutional layer , in this case the maxpool layer
        final_conv_layer = model.layers[-3]
        
        #Get output of the selected layers given input image
        get_output = K.function([model.layers[0].input], [final_conv_layer.output, model.layers[-1].output])
        [conv_outputs, predictions] = get_output([batch_img])
        
        #Pick the conv_outputs of the one and only image
        conv_outputs = conv_outputs[0, :, :, :]

        #Create the class activation map.
        cam = np.zeros(dtype = np.float32, shape = conv_outputs.shape[0:2])
        
        #target_class defines the weight to select when working on activation maps (0 for cats, 1 for dogs)
        for i, w in enumerate(class_weights[:,target_class]):
            cam += w * conv_outputs[:, :,i]
            
        
        cam /= np.max(cam)
        cam = cv2.resize(cam, (height, width))
        heatmap = cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET)
        heatmap[np.where(cam < 0.2)] = 0
        img = heatmap*0.5 + img
        
        #Save CAM
        cv2.imwrite(output_path, img)

### Defining and loading model

In [10]:
# dimensions of our images.
img_width, img_height = 150, 150

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
    
#Helper functions to define model
def global_average_pooling(x):
    return K.mean(x, axis = (1, 2))

def global_average_pooling_shape(input_shape):
    return input_shape[0:2]

#Defining Model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape,activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Lambda(global_average_pooling, 
              output_shape=global_average_pooling_shape))
model.add(Dense(2, activation = 'softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


#Since we have a trained model, load the model weights
model_path = 'weights/weights.50-0.43.h5' 
model.load_weights(model_path)

#Dog CAM
path = 'data/train/dogs'
images = os.listdir(path)
index = np.random.randint(0,len(images))
img_path = os.path.join(path,images[index])

visualize_class_activation_map(model,img_path,'CAM/dog.png',1)

#Cat CAM
path = 'data/train/cats'
images = os.listdir(path)
index = np.random.randint(0,len(images))
img_path = os.path.join(path,images[index])

visualize_class_activation_map(model,img_path,'CAM/cat.png',0)


### CAM for randomly selected dog image

We can see our network is looking at the image and paying attention to its face, and the tail region to make a decision 

![title](CAM/dog2.png)

### CAM for randomly selected cat image

We can see our network is looking at the image and paying attention to its face region to make a decision 

![title](CAM/cat2.png)